In [64]:
from sqlalchemy import create_engine
import pandas as pd
from sqlalchemy import Table, Column, Integer, String, ForeignKey, MetaData
from sqlalchemy.orm import declarative_base, relationship, Session,sessionmaker
import pymysql
import cryptography 
import numpy as np

username = "root"
password = "R1e3z8a2/"
host = "localhost"   
port = 3306             
database = "nba"

engine = create_engine(f"mysql+pymysql://{username}:{password}@{host}:{port}/{database}", connect_args={"connect_timeout": 60})
metadata = MetaData()
Base = declarative_base()
session = Session(engine)

In [87]:
import ast

In [ ]:
class Player(Base):
    __tablename__ = "players"
    id = Column(String(64), primary_key= True)
    name = Column(String(64), unique= True)
    weight = Column(Integer)
    height = Column(Integer)
    exp = Column(Integer)
    age = Column(Integer)
    
    fact = relationship("Fact", back_populates="player")
    player_position = relationship("PlayerPosition", back_populates="player")
    team = relationship("PlayerTeam",back_populates="player")

In [66]:
class Season(Base):
    __tablename__ ="seasons"
    id = Column(Integer,primary_key=True,autoincrement=True)
    season = Column(Integer,unique=True)
    fact = relationship("Fact",back_populates="season")


In [ ]:
class PlayerGroup(Base):
    __tablename__ ="player_group"
    id = Column(Integer,primary_key=True,autoincrement=True)
    player_group = Column(String(64),unique=True)
    fact = relationship("Fact",back_populates="player_group")

In [ ]:
class Position(Base):
    __tablename__ ="position"
    id = Column(Integer,primary_key=True,autoincrement=True)
    position = Column(String(64),unique=True)
    player_position = relationship("PlayerPosition",back_populates="position")



In [69]:
class Fact(Base):
    __tablename__ ="fact"
    id = Column(Integer,primary_key=True)
    player_id = Column(String(64),ForeignKey("players.id"))
    season_id = Column(Integer, ForeignKey("seasons.id"))
    group_id = Column(Integer, ForeignKey("player_group.id"))
    rank = Column(Integer)
    points = Column(Integer)
    
    player = relationship("Player",back_populates="fact")
    season = relationship("Season",back_populates="fact")
    player_group = relationship("PlayerGroup",back_populates="fact")

In [ ]:
class PlayerPosition(Base):
    __tablename__ ="player_position"
    id = Column(Integer,primary_key=True,autoincrement=True)
    player_id = Column(String(64),ForeignKey("players.id"))
    position_id = Column(Integer,ForeignKey("position.id"))
    player = relationship("Player",back_populates="player_position")
    position = relationship("Position",back_populates="player_position")

In [ ]:
class Team(Base):
    __tablename__="team"
    id = Column(Integer,primary_key=True,autoincrement=True) 
    team_name = Column(String(64), unique= True)
    players = relationship("PlayerTeam",back_populates="team")

In [72]:
class PlayerTeam(Base):
    __tablename__="player_team"
    id = Column(Integer,primary_key=True)
    team_id = Column(Integer,ForeignKey("team.id"))
    player_id = Column(String(64),ForeignKey("players.id"))
    player = relationship("Player",back_populates="team")
    team = relationship("Team",back_populates="players")

In [73]:
Base.metadata.drop_all(engine)
Base.metadata.create_all(engine)

In [74]:
import requests
from bs4 import BeautifulSoup
import time, random
import re
import pandas as pd

from selenium.webdriver.support.expected_conditions import none_of

headers = {
    "User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/140.0.0.0 Safari/537.36",
    "Accept-Language": "en-US,en;q=0.9",
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8",
    "Referer": "https://www.google.com/",
    "DNT": "1",
    "Connection": "keep-alive",
}

In [75]:

year=2026
years_list=[]
list_href_mvp=[]
list_href_player=[]
base_url = "https://www.basketball-reference.com"
mvp_url="/awards/mvp.html"
full_url=base_url+mvp_url

response = requests.get(full_url, headers=headers)
response.encoding = "utf-8"
print(response.status_code)
soup = BeautifulSoup(response.text, "html.parser")

rows = soup.select("tbody tr")
count=0

for row in rows:
    if count== 7:
        break
    cells = row.select("td")
    third_td = cells[2]
    link_tag = third_td.select_one("a")
    list_href_mvp.append(link_tag["href"])
    # print(link_tag["href"])



    count += 1
for best_mvp_each_year_url in list_href_mvp:
    full_url_mvp_each_year = base_url + best_mvp_each_year_url

    response = requests.get(full_url_mvp_each_year, headers=headers)
    response.encoding = "utf-8"
    print(response.status_code)
    time.sleep(random.uniform(3,5))
    soup_mvp_person = BeautifulSoup(response.text, "html.parser")

    players = soup_mvp_person.select("#div_mvp tbody tr")
    year=year-1
    years_list.append(year)
    for player in players:
        player_cell = player.select("td")
        link_player = player_cell[0].select_one("a")

        age=player_cell[1].text
        age_int=int(age)

        rank = player.select("th")
        player_rank = rank[0].text
        numbers = "".join(re.findall(r"\d+", player_rank))
        number_int = int(numbers)
        list_href_player.append({"year": year, "url_player": link_player["href"], "age": age_int, "player_rank": number_int})

200
200
200
200
200
200
200
200


In [76]:

positions = []
records=[]
height_cm=0
weight_kg=0
experience_years=0
cleaned=None
player_name=None
team_name=None

for player in list_href_player:
    response = requests.get(base_url+player["url_player"],headers=headers)
    response.encoding = "utf-8"
    info_player_soup = BeautifulSoup(response.text, "html.parser")


    url = player["url_player"]  # just a string
    player_id = re.search(r'([^/]+)\.html$', url).group(1)


    player_name_elem = info_player_soup.select_one("#meta h1")
    player_name = player_name_elem.get_text(strip=True) if player_name_elem else None

    final_clean_positions = []
    position_elem = info_player_soup.select_one("#meta p:has(strong:-soup-contains('Position'))")

    text = position_elem.get_text(" ", strip=True)
    position = text.split("Shoots:")[0].replace("Position:", "").strip()
    clean_position = position.replace(", and", ",")
    clean2_position = clean_position.replace(" and ", ",")
    positions = [p.strip() for p in clean2_position.split(",") if p.strip()]
    for p in positions:
        cleaned = p.strip()
        cleaned = cleaned.replace("▪", "")
        cleaned = cleaned.strip()
        final_clean_positions.append(cleaned)

    team_strong = info_player_soup.select_one("#meta strong:-soup-contains('Team')")
    if team_strong:
        team_link_tag = team_strong.find_next("a")
        if team_link_tag:
            team_name = team_link_tag.get_text(strip=True)
    else:
        team_name = None



    exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Experience')")
    if exp_strong:
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))

    else:
        exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Career Length')")
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))




    height_weight_elems = info_player_soup.select_one("#meta p:has(span:-soup-contains('lb'))")
    text = height_weight_elems.get_text(" ", strip=True)
    match = re.search(r"\((\d+)cm,\s*(\d+)kg\)", text)
    height_cm = int(match.group(1))
    weight_kg = int(match.group(2))

    records.append({
        "player_id": player_id,
        "player_name": player_name,
        "age": player["age"],
        "rank": player["player_rank"],
        "season": player["year"],
        "height_cm": height_cm,
        "weight_kg": weight_kg,
        "experience_years": experience_years,
        "positions": final_clean_positions,
        "team": team_name,
        "total_point": None,
        "groupName":"MVP"
    })

    time.sleep(random.uniform(3,5))
df_mvp = pd.DataFrame(records)
print(df_mvp.head())


   player_id              player_name  age  rank  season  height_cm  \
0  gilgesh01  Shai Gilgeous-Alexander   26     1    2025        198   
1  jokicni01             Nikola Jokić   29     2    2025        211   
2  antetgi01    Giannis Antetokounmpo   30     3    2025        211   
3  tatumja01             Jayson Tatum   26     4    2025        203   
4  mitchdo01         Donovan Mitchell   28     5    2025        190   

   weight_kg  experience_years  \
0         90                 7   
1        128                10   
2        109                12   
3         95                 8   
4         97                 8   

                                           positions                   team  \
0                      [Point Guard, Shooting Guard]  Oklahoma City Thunder   
1                                           [Center]         Denver Nuggets   
2  [Power Forward, Small Forward, Point Guard, Sh...        Milwaukee Bucks   
3                     [Small Forward, Power Forward]

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [77]:
years=["2025","2024","2023","2022","2021","2020"]
list_href_player=[]
base_url = "https://www.basketball-reference.com"
for year in years:

    best_player_url="/leagues/NBA_" +year+"_totals.html"
    full_url=base_url+best_player_url

    response = requests.get(full_url, headers=headers)
    response.encoding = "utf-8"
    print(response.status_code)
    time.sleep(random.uniform(3,5))
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.select("tbody tr")

    year_int=int(year)
    for row in rows:
        rank = row.select("th")
        player_rank = rank[0].text
        numbers = "".join(re.findall(r"\d+", player_rank))
        number_int = int(numbers)

        player_cell = row.select("td")
        link_player = player_cell[0].select_one("a")
        age=player_cell[1].text
        age_int=int(age)
        total_point=player_cell[28].text
        total_point_int=int(total_point)
        print(link_player["href"])
        print(total_point_int)
        list_href_player.append({"year": year_int, "url_player": link_player["href"], "age": age_int, "player_rank": number_int,"total_point": total_point_int})
        if number_int==25:
            break

200
/players/g/gilgesh01.html
2484
/players/e/edwaran01.html
2177
/players/j/jokicni01.html
2071
/players/a/antetgi01.html
2036
/players/t/tatumja01.html
1932
/players/b/bookede01.html
1923
/players/y/youngtr01.html
1841
/players/h/herroty01.html
1840
/players/c/cunnica01.html
1830
/players/h/hardeja01.html
1802
/players/t/townska01.html
1759
/players/l/lavinza01.html
1724
/players/l/lavinza01.html
1007
/players/l/lavinza01.html
717
/players/g/greenja05.html
1723
/players/c/curryst01.html
1718
/players/d/derozde01.html
1710
/players/j/jamesle01.html
1710
/players/m/mitchdo01.html
1701
/players/b/brunsja01.html
1690
/players/d/duranke01.html
1647
/players/j/jacksja02.html
1641
/players/s/siakapa01.html
1578
/players/g/garlada01.html
1544
/players/w/whiteco01.html
1509
/players/w/willija06.html
1490
/players/r/reaveau01.html
1475
200
/players/d/doncilu01.html
2370
/players/g/gilgesh01.html
2254
/players/a/antetgi01.html
2222
/players/b/brunsja01.html
2212
/players/j/jokicni01.html
2085
/

In [78]:

positions = []
records=[]
height_cm=0
weight_kg=0
experience_years=0
cleaned=None
player_name=None
team_name=None
for player in list_href_player:
    response = requests.get(base_url+player["url_player"],headers=headers)
    response.encoding = "utf-8"
    info_player_soup = BeautifulSoup(response.text, "html.parser")

    url_player = player["url_player"]  # just a string
    player_id = re.search(r'([^/]+)\.html$', url_player).group(1)

    player_name_elem = info_player_soup.select_one("#meta h1")
    player_name = player_name_elem.get_text(strip=True) if player_name_elem else None
    print(player_name)

    final_clean_positions = []
    position_elem = info_player_soup.select_one("#meta p:has(strong:-soup-contains('Position'))")

    text = position_elem.get_text(" ", strip=True)
    position = text.split("Shoots:")[0].replace("Position:", "").strip()
    clean_position = position.replace(", and", ",")
    clean2_position = clean_position.replace(" and ", ",")
    positions = [p.strip() for p in clean2_position.split(",") if p.strip()]
    for p in positions:
        cleaned = p.strip()
        cleaned = cleaned.replace("▪", "")
        cleaned = cleaned.strip()
        final_clean_positions.append(cleaned)

    team_strong = info_player_soup.select_one("#meta strong:-soup-contains('Team')")
    if team_strong:
        team_link_tag = team_strong.find_next("a")
        if team_link_tag:
            team_name = team_link_tag.get_text(strip=True)
    else:
        team_name = None



    exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Experience')")
    if exp_strong:
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))

    else:
        exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Career Length')")
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))




    height_weight_elems = info_player_soup.select_one("#meta p:has(span:-soup-contains('lb'))")
    text = height_weight_elems.get_text(" ", strip=True)
    match = re.search(r"\((\d+)cm,\s*(\d+)kg\)", text)
    height_cm = int(match.group(1))
    weight_kg = int(match.group(2))

    records.append({
        "player_id": player_id,
        "player_name": player_name,
        "age": player["age"],
        "rank": player["player_rank"],
        "season": player["year"],
        "height_cm": height_cm,
        "weight_kg": weight_kg,
        "experience_years": experience_years,
        "positions": final_clean_positions,
        "team": team_name,
        "total_point": player["total_point"],
        "groupName":"Total Stat"
    })

    time.sleep(random.uniform(3,5))
df_BP = pd.DataFrame(records)
print(df_BP.head())

Shai Gilgeous-Alexander
Anthony Edwards
Nikola Jokić
Giannis Antetokounmpo
Jayson Tatum
Devin Booker
Trae Young
Tyler Herro
Cade Cunningham
James Harden
Karl-Anthony Towns
Zach LaVine
Zach LaVine
Zach LaVine
Jalen Green
Stephen Curry
DeMar DeRozan
LeBron James
Donovan Mitchell
Jalen Brunson
Kevin Durant
Jaren Jackson Jr.
Pascal Siakam
Darius Garland
Coby White
Jalen Williams
Austin Reaves
Luka Dončić
Shai Gilgeous-Alexander
Giannis Antetokounmpo
Jalen Brunson
Nikola Jokić
Anthony Edwards
Kevin Durant
Jayson Tatum
De'Aaron Fox
Stephen Curry
DeMar DeRozan
Anthony Davis
Devin Booker
LeBron James
Tyrese Maxey
Paolo Banchero
Damian Lillard
Dejounte Murray
Pascal Siakam
Pascal Siakam
Pascal Siakam
Paul George
Kawhi Leonard
Jaylen Brown
Jalen Green
Mikal Bridges
Zion Williamson
Jayson Tatum
Joel Embiid
Luka Dončić
Shai Gilgeous-Alexander
Giannis Antetokounmpo
Anthony Edwards
Julius Randle
Donovan Mitchell
Trae Young
Zach LaVine
Damian Lillard
De'Aaron Fox
DeMar DeRozan
Jaylen Brown
Pascal Sia

In [79]:
df_BP = df_BP.drop_duplicates(subset=["season", "rank"], keep="first").reset_index(drop=True)
df_BP

,player_id,player_name,age,rank,season,height_cm,weight_kg,experience_years,positions,team,total_point,groupName
0,gilgesh01,Shai Gilgeous-Alexander,26,1,2025,198,90,7,"[Point Guard, Shooting Guard]",Oklahoma City Thunder,2484,Total Stat
1,edwaran01,Anthony Edwards,23,2,2025,193,102,5,[Shooting Guard],Minnesota Timberwolves,2177,Total Stat
2,jokicni01,Nikola Jokić,29,3,2025,211,128,10,[Center],Denver Nuggets,2071,Total Stat
3,antetgi01,Giannis Antetokounmpo,30,4,2025,211,109,12,"[Power Forward, Small Forward, Point Guard, Sh...",Milwaukee Bucks,2036,Total Stat
4,tatumja01,Jayson Tatum,26,5,2025,203,95,8,"[Small Forward, Power Forward]",Boston Celtics,1932,Total Stat
...,...,...,...,...,...,...,...,...,...,...,...,...
145,siakapa01,Pascal Siakam,25,21,2020,203,104,9,[Power Forward],Indiana Pacers,1371,Total Stat
146,sextoco01,Collin Sexton,21,22,2020,190,86,7,"[Shooting Guard, Point Guard]",Charlotte Hornets,1349,Total Stat
147,gilgesh01,Shai Gilgeous-Alexander,21,23,2020,198,90,7,"[Point Guard, Shooting Guard]",Oklahoma City Thunder,1331,Total Stat
148,warretj01,T.J. Warren,26,24,2020,203,99,9,"[Small Forward, Power Forward]",None,1326,Total Stat


---------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [80]:
list_href_player=[]
year_list=[2025,2024,2023,2022,2021,2020]
i=0
url_champion=["https://www.basketball-reference.com/teams/OKC/2025.html","https://www.basketball-reference.com/teams/BOS/2024.html","https://www.basketball-reference.com/teams/DEN/2023.html","https://www.basketball-reference.com/teams/GSW/2022.html","https://www.basketball-reference.com/teams/MIL/2021.html","https://www.basketball-reference.com/teams/LAL/2020.html"]
for url in url_champion:

    response = requests.get(url, headers=headers)
    response.encoding = "utf-8"
    print(response.status_code)
    time.sleep(random.uniform(3,5))
    soup = BeautifulSoup(response.text, "html.parser")
    rows = soup.select("table#per_game_stats tbody tr")

    for row in rows:

        player_cell = row.select("td")
        link_player = player_cell[0].select_one("a")
        age=player_cell[1].text
        age_int=int(age)

        print(link_player["href"])

        list_href_player.append({"year":year_list[i], "url_player": link_player["href"], "age": age_int, "player_rank":None,"total_point":None})

    i=i+1

200
/players/g/gilgesh01.html
/players/w/willija06.html
/players/d/dortlu01.html
/players/h/harteis01.html
/players/w/wallaca01.html
/players/h/holmgch01.html
/players/w/wiggiaa01.html
/players/j/joeis01.html
/players/c/carusal01.html
/players/w/willija07.html
/players/m/mitchaj01.html
/players/w/willike04.html
/players/d/diengou01.html
/players/j/jonesdi01.html
/players/c/carlsbr01.html
/players/d/ducasal01.html
/players/f/flaglad01.html
/players/l/leonsma01.html
/players/r/reeseal01.html
200
/players/t/tatumja01.html
/players/b/brownja02.html
/players/h/holidjr01.html
/players/w/whitede01.html
/players/p/porzikr01.html
/players/h/horfoal01.html
/players/p/pritcpa01.html
/players/h/hausesa01.html
/players/k/kornelu01.html
/players/t/tillmxa01.html
/players/q/quetane01.html
/players/b/brissos01.html
/players/m/mykhasv01.html
/players/w/walshjo01.html
/players/p/peterdr01.html
/players/s/sprinja01.html
/players/b/bantoda01.html
/players/s/stevela01.html
/players/d/davisjd01.html
200
/pl

In [81]:

positions = []
records=[]
height_cm=0
weight_kg=0
experience_years=0
cleaned=None
player_name=None
team_name=None
base_url = "https://www.basketball-reference.com"
for player in list_href_player:
    response = requests.get(base_url+player["url_player"],headers=headers)
    response.encoding = "utf-8"
    info_player_soup = BeautifulSoup(response.text, "html.parser")

    url_player = player["url_player"]  # just a string
    player_id = re.search(r'([^/]+)\.html$', url_player).group(1)

    player_name_elem = info_player_soup.select_one("#meta h1")
    player_name = player_name_elem.get_text(strip=True) if player_name_elem else None
    print(player_name)

    final_clean_positions = []
    position_elem = info_player_soup.select_one("#meta p:has(strong:-soup-contains('Position'))")

    text = position_elem.get_text(" ", strip=True)
    position = text.split("Shoots:")[0].replace("Position:", "").strip()
    clean_position = position.replace(", and", ",")
    clean2_position = clean_position.replace(" and ", ",")
    positions = [p.strip() for p in clean2_position.split(",") if p.strip()]
    for p in positions:
        cleaned = p.strip()
        cleaned = cleaned.replace("▪", "")
        cleaned = cleaned.strip()
        final_clean_positions.append(cleaned)

    team_strong = info_player_soup.select_one("#meta strong:-soup-contains('Team')")
    if team_strong:
        team_link_tag = team_strong.find_next("a")
        if team_link_tag:
            team_name = team_link_tag.get_text(strip=True)
    else:
        team_name = None



    exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Experience')")
    if exp_strong:
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))

    else:
        exp_strong = info_player_soup.select_one("#meta strong:-soup-contains('Career Length')")
        experience_text = exp_strong.next_sibling.strip()
        match = re.search(r"\d+", experience_text)
        experience_years = int(match.group(0))




    height_weight_elems = info_player_soup.select_one("#meta p:has(span:-soup-contains('lb'))")
    text = height_weight_elems.get_text(" ", strip=True)
    match = re.search(r"\((\d+)cm,\s*(\d+)kg\)", text)
    height_cm = int(match.group(1))
    weight_kg = int(match.group(2))

    records.append({
        "player_id": player_id,
        "player_name": player_name,
        "age": player["age"],
        "rank": player["player_rank"],
        "season": player["year"],
        "height_cm": height_cm,
        "weight_kg": weight_kg,
        "experience_years": experience_years,
        "positions": final_clean_positions,
        "team": team_name,
        "total_point": player["total_point"],
        "groupName":"League Champion"
    })

    time.sleep(random.uniform(3,5))
df_CP = pd.DataFrame(records)

Shai Gilgeous-Alexander
Jalen Williams
Luguentz Dort
Isaiah Hartenstein
Cason Wallace
Chet Holmgren
Aaron Wiggins
Isaiah Joe
Alex Caruso
Jaylin Williams
Ajay Mitchell
Kenrich Williams
Ousmane Dieng
Dillon Jones
Branden Carlson
Alex Ducas
Adam Flagler
Malevy Leons
Alex Reese
Jayson Tatum
Jaylen Brown
Jrue Holiday
Derrick White
Kristaps Porziņģis
Al Horford
Payton Pritchard
Sam Hauser
Luke Kornet
Xavier Tillman Sr.
Neemias Queta
Oshae Brissett
Svi Mykhailiuk
Jordan Walsh
Drew Peterson
Jaden Springer
Dalano Banton
Lamar Stevens
JD Davison
Nikola Jokić
Jamal Murray
Kentavious Caldwell-Pope
Aaron Gordon
Michael Porter Jr.
Bruce Brown
Reggie Jackson
Jeff Green
Bones Hyland
Christian Braun
DeAndre Jordan
Vlatko Čančar
Zeke Nnaji
Thomas Bryant
Ish Smith
Davon Reed
Peyton Watson
Jack White
Stephen Curry
Andrew Wiggins
Jordan Poole
Klay Thompson
Draymond Green
Otto Porter Jr.
Kevon Looney
Damion Lee
Andre Iguodala
Gary Payton II
Jonathan Kuminga
Nemanja Bjelica
Juan Toscano-Anderson
Moses Moody


In [82]:
df_final = pd.concat([df_BP,df_mvp, df_CP], ignore_index=True)



#df_final.to_csv("df_final.csv", index=False)
df_final

,player_id,player_name,age,rank,season,height_cm,weight_kg,experience_years,positions,team,total_point,groupName
0,gilgesh01,Shai Gilgeous-Alexander,26,1,2025,198,90,7,"[Point Guard, Shooting Guard]",Oklahoma City Thunder,2484,Total Stat
1,edwaran01,Anthony Edwards,23,2,2025,193,102,5,[Shooting Guard],Minnesota Timberwolves,2177,Total Stat
2,jokicni01,Nikola Jokić,29,3,2025,211,128,10,[Center],Denver Nuggets,2071,Total Stat
3,antetgi01,Giannis Antetokounmpo,30,4,2025,211,109,12,"[Power Forward, Small Forward, Point Guard, Sh...",Milwaukee Bucks,2036,Total Stat
4,tatumja01,Jayson Tatum,26,5,2025,203,95,8,"[Small Forward, Power Forward]",Boston Celtics,1932,Total Stat
...,...,...,...,...,...,...,...,...,...,...,...,...
345,danietr01,Troy Daniels,28,None,2020,193,90,7,[Shooting Guard],None,None,League Champion
346,cacokde01,Devontae Cacok,23,None,2020,201,108,3,"[Power Forward, Center]",None,None,League Champion
347,dudleja01,Jared Dudley,34,None,2020,198,107,14,"[Power Forward, Small Forward, Shooting Guard]",None,None,League Champion
348,antetko01,Kostas Antetokounmpo,22,None,2020,208,90,3,"[Power Forward, Small Forward]",None,None,League Champion


In [ ]:
season_table = df_final['season'].unique()
season_table = np.sort(season_table)
for s in season_table:
    season_o = Season(season=int(s))
    session.add(season_o)
session.commit()

In [85]:
group_table = df_final['groupName'].unique()
group_table
for g in group_table:
    player_group_o = PlayerGroup(player_group=str(g))
    session.add(player_group_o)
session.commit()

In [93]:
position_table = df_final[['positions']].copy()
position_table = position_table.explode('positions')
position_table = position_table['positions'].unique()


array(['Point Guard', 'Shooting Guard', 'Center', 'Power Forward',
       'Small Forward'], dtype=object)

In [94]:
for p in position_table:
    position_o = Position(position=str(p))
    session.add(position_o)
session.commit()

In [95]:
player_table = df_final[['player_id','player_name','age','height_cm','weight_kg','experience_years']]
player_table = player_table.drop_duplicates(subset="player_id").reset_index(drop=True)
player_table

,player_id,player_name,age,height_cm,weight_kg,experience_years
0,gilgesh01,Shai Gilgeous-Alexander,26,198,90,7
1,edwaran01,Anthony Edwards,23,193,102,5
2,jokicni01,Nikola Jokić,29,211,128,10
3,antetgi01,Giannis Antetokounmpo,30,211,109,12
4,tatumja01,Jayson Tatum,26,203,95,8
...,...,...,...,...,...,...
160,danietr01,Troy Daniels,28,193,90,7
161,cacokde01,Devontae Cacok,23,201,108,3
162,dudleja01,Jared Dudley,34,198,107,14
163,antetko01,Kostas Antetokounmpo,22,208,90,3


In [97]:
for _,pl in player_table.iterrows():
    player_o = Player(id=pl['player_id'], name = pl['player_name'],weight = pl['weight_kg'],
                    height = pl['height_cm'],exp = pl['experience_years'],age = pl['age'])

    session.add(player_o)
session.commit()

In [106]:
position_player_table = df_final[['player_id','positions']]
position_player_table  = position_player_table.drop_duplicates(subset="player_id").reset_index(drop=True)
position_player_table = position_player_table.explode("positions")
position_table_sql = pd.read_sql('SELECT id, position FROM position', con=engine)
position_player_table = position_player_table.merge(position_table_sql, how='left', left_on='positions', right_on='position')
position_player_table = position_player_table[['player_id','id']]
position_player_table

,player_id,id
0,gilgesh01,1
1,gilgesh01,2
2,edwaran01,2
3,jokicni01,3
4,antetgi01,4
...,...,...
268,dudleja01,5
269,dudleja01,2
270,antetko01,4
271,antetko01,5


In [108]:
for _,pp in position_player_table.iterrows():
    position_player_o = PlayerPosition(player_id=pp['player_id'], position_id = pp['id'])

    session.add(position_player_o)
session.commit()

In [113]:
fact_table = df_final[['player_id','season','groupName','rank','total_point']]
seasons_table_sql = pd.read_sql('SELECT id, season FROM seasons', con=engine)
fact_table = fact_table.merge(seasons_table_sql, how='left', left_on='season', right_on='season')
fact_table = fact_table.rename(columns={'id':'season_id'})
group_table_sql = pd.read_sql('SELECT id, player_group FROM player_group', con=engine)
fact_table = fact_table.merge(group_table_sql, how='left', left_on='groupName', right_on='player_group')
fact_table = fact_table[['player_id','season_id','id','rank','total_point']]
fact_table


,player_id,season_id,id,rank,total_point
0,gilgesh01,7,1,1,2484
1,edwaran01,7,1,2,2177
2,jokicni01,7,1,3,2071
3,antetgi01,7,1,4,2036
4,tatumja01,7,1,5,1932
...,...,...,...,...,...
345,danietr01,2,3,None,None
346,cacokde01,2,3,None,None
347,dudleja01,2,3,None,None
348,antetko01,2,3,None,None


In [118]:
for _,f in fact_table.iterrows():
    fact_o = Fact(player_id=f['player_id'], season_id = f['season_id'],group_id = f['id'],
                  rank = f['rank'],points=f['total_point'])

    session.add(fact_o)
session.commit()